# Pydantic-based Mesh Example

This notebook demonstrates how to use the new Pydantic-based Mesh class in meshly. It covers:

1. Creating custom Mesh subclasses with additional attributes
2. Working with numpy arrays in Pydantic models
3. Encoding and decoding meshes to/from zip files
4. Optimizing meshes with the built-in optimization methods

In [1]:
from pathlib import Path

import numpy as np

# Import the Mesh class
from meshly import Mesh, Array, Resource
from pydantic import Field

## 1. Creating a Custom Mesh Subclass

One of the key benefits of the new Pydantic-based Mesh class is the ability to create custom subclasses with additional attributes. Let's create a `TexturedMesh` class that adds texture coordinates and normals.

In [2]:
from typing import ClassVar
from meshly import Packable
from pydantic import BaseModel, ConfigDict


class MaterialProperties(BaseModel):
    """Material properties with numpy arrays - demonstrates BaseModel in dict edge case."""
    model_config = ConfigDict(arbitrary_types_allowed=True)

    name: str = Field(..., description="Material name")
    diffuse: Array = Field(..., description="Diffuse color array")
    specular: Array = Field(..., description="Specular color array")
    shininess: float = Field(32.0, description="Shininess value")


class PhysicsProperties(Packable):
    """Physics properties as a nested Packable - demonstrates Packable field support."""
    _self_contained: ClassVar[bool] = True
    mass: float = Field(1.0, description="Object mass")
    friction: float = Field(0.5, description="Friction coefficient")
    # Arrays in nested Packable are encoded/decoded using the Packable's own encode/decode
    inertia_tensor: Array = Field(..., description="3x3 inertia tensor")
    collision_points: Array = Field(..., description="Collision sample points")


class TexturedMesh(Mesh):
    """
    A mesh with texture coordinates and normals.
    
    This demonstrates how to create a custom Mesh subclass with additional
    numpy array attributes that will be automatically encoded/decoded.
    """
    # Add texture coordinates and normals as additional numpy arrays
    texture_coords: Array = Field(..., description="Texture coordinates")
    normals: Array | None = Field(None, description="Vertex normals")

    # Add non-array attributes
    material_name: str = Field("default", description="Material name")
    tags: list[str] = Field(default_factory=list, description="Tags for the mesh")

    # Resource field - file that gets serialized with the mesh
    material_file: Resource | None = Field(None, description="MTL material file")

    # Dictionary containing nested dictionaries with arrays
    material_data: dict[str, dict[str, Array]] = Field(
        default_factory=dict,
        description="Nested dictionary structure with arrays"
    )

    physics: PhysicsProperties | None = Field(
        None,
        description="Physics properties as a nested Packable"
    )

    material_colors: dict[str, str] = Field(
        default_factory=dict,
        description="Dictionary with non-array values"
    )

    # Dictionary containing BaseModel instances with numpy arrays
    # This demonstrates the edge case of dict[str, BaseModel] where BaseModel has arrays
    materials: dict[str, MaterialProperties] = Field(
        default_factory=dict,
        description="Dictionary of material name to MaterialProperties (BaseModel with arrays)"
    )

## 2. Creating a Mesh Instance

Now let's create a simple cube mesh with texture coordinates and normals.

In [3]:
# Create vertices for a cube
vertices = np.array([
    [-0.5, -0.5, -0.5],  # 0: bottom-left-back
    [0.5, -0.5, -0.5],   # 1: bottom-right-back
    [0.5, 0.5, -0.5],    # 2: top-right-back
    [-0.5, 0.5, -0.5],   # 3: top-left-back
    [-0.5, -0.5, 0.5],   # 4: bottom-left-front
    [0.5, -0.5, 0.5],    # 5: bottom-right-front
    [0.5, 0.5, 0.5],     # 6: top-right-front
    [-0.5, 0.5, 0.5]     # 7: top-left-front
], dtype=np.float32)

# Create indices for the cube
indices = np.array([
    [0, 1, 2, 2, 3, 0],  # back face
    [1, 5, 6, 6, 2, 1],  # right face
    [5, 4, 7, 7, 6, 5],  # front face
    [4, 0, 3, 3, 7, 4],  # left face
    [3, 2, 6, 6, 7, 3],  # top face
    [4, 5, 1, 1, 0, 4]   # bottom face
], dtype=np.uint32)

# Create texture coordinates (one for each vertex)
texture_coords = np.array([
    [0.0, 0.0],  # 0
    [1.0, 0.0],  # 1
    [1.0, 1.0],  # 2
    [0.0, 1.0],  # 3
    [0.0, 0.0],  # 4
    [1.0, 0.0],  # 5
    [1.0, 1.0],  # 6
    [0.0, 1.0]   # 7
], dtype=np.float32)

# Create normals (one for each vertex)
normals = np.array([
    [0.0, 0.0, -1.0],  # 0: back
    [0.0, 0.0, -1.0],  # 1: back
    [0.0, 0.0, -1.0],  # 2: back
    [0.0, 0.0, -1.0],  # 3: back
    [0.0, 0.0, 1.0],   # 4: front
    [0.0, 0.0, 1.0],   # 5: front
    [0.0, 0.0, 1.0],   # 6: front
    [0.0, 0.0, 1.0]    # 7: front
], dtype=np.float32)

# Create MaterialProperties instances (BaseModel with numpy arrays)
cube_material = MaterialProperties(
    name="cube_material",
    diffuse=np.array([1.0, 0.5, 0.31], dtype=np.float32),
    specular=np.array([0.5, 0.5, 0.5], dtype=np.float32),
    shininess=32.0
)

secondary_material = MaterialProperties(
    name="secondary_material",
    diffuse=np.array([0.2, 0.8, 0.2], dtype=np.float32),
    specular=np.array([0.3, 0.3, 0.3], dtype=np.float32),
    shininess=16.0
)

# Create PhysicsProperties instance (nested Packable)
physics = PhysicsProperties(
    mass=2.5,
    friction=0.7,
    inertia_tensor=np.eye(3, dtype=np.float32) * 0.1,  # 3x3 identity scaled
    collision_points=np.array([
        [-0.5, -0.5, -0.5],
        [0.5, 0.5, 0.5],
        [0.0, 0.0, 0.0]
    ], dtype=np.float32)
)

# Create the textured mesh with a material file resource
# ResourceRef resolves relative paths to absolute at creation time
mesh = TexturedMesh(
    vertices=vertices,
    indices=indices,
    texture_coords=texture_coords,
    normals=normals,
    physics=physics,
    material_name="cube_material",
    tags=["cube", "example"],
    material_file="examples/resources/cube_material.mtl",  # Relative path - auto-resolved to absolute
    material_data={
        "cube_material": {
            "diffuse": np.array([1.0, 0.5, 0.31], dtype=np.float32),
            "specular": np.array([0.5, 0.5, 0.5], dtype=np.float32),
            "shininess": np.array([32.0], dtype=np.float32)
        }
    },
    material_colors={
        "cube_material": "#FF7F50"
    },
    # dict[str, BaseModel] with numpy arrays inside
    materials={
        "cube_material": cube_material,
        "secondary_material": secondary_material
    },
)

print(f"Mesh created with {mesh.vertex_count} vertices and {mesh.index_count} indices")
print(f"Material name: {mesh.material_name}")
print(f"Tags: {mesh.tags}")
print(f"Material file: {mesh.material_file}")  # Shows resolved absolute path
print(f"Materials (BaseModel dict): {list(mesh.materials.keys())}")

Mesh created with 8 vertices and 36 indices
Material name: cube_material
Tags: ['cube', 'example']
Material file: examples/resources/cube_material.mtl
Materials (BaseModel dict): ['cube_material', 'secondary_material']


## 3. Optimizing the Mesh

The Mesh class provides several optimization methods that can be used to improve rendering performance.

In [4]:
# Optimize the mesh for vertex cache
vertex_cache_optimized_mesh = mesh.optimize_vertex_cache()
print("Optimized for vertex cache")

# Optimize the mesh for overdraw
overdraw_optimized_mesh = mesh.optimize_overdraw()
print("Optimized for overdraw")

# Optimize the mesh for vertex fetch
vertex_fetch_optimized = mesh.optimize_vertex_fetch()
print("Optimized for vertex fetch")

Optimized for vertex cache
Optimized for overdraw
Optimized for vertex fetch


## 4. Encoding and Saving the Mesh

The Mesh class provides methods for encoding the mesh and saving it to a zip file.

In [10]:
# Save the mesh to a zip file
zip_path = Path("textured_cube.zip")
mesh.save_to_zip(zip_path)
assert zip_path.exists()
print(f"Saved mesh to {zip_path} has {mesh.vertex_count} vertices and {mesh.index_count} indices")


decoded_mesh = Mesh.decode(mesh.encode())
safe_deconded_mesh = Mesh.decode_from_schema(mesh.encode(), as_model=True)
print(f"Decoded mesh has {decoded_mesh.vertex_count} vertices and {decoded_mesh.index_count} indices")

Saved mesh to textured_cube.zip has 8 vertices and 36 indices
Decoded mesh has 8 vertices and 36 indices


In [6]:
safe_deconded_mesh

TexturedMesh(cell_types=array([13, 13, 13, 13, 13, 13], dtype=uint32), dim=3, index_sizes=array([6, 6, 6, 6, 6, 6], dtype=uint32), indices=array([0, 1, 2, 2, 3, 0, 1, 5, 6, 6, 2, 1, 5, 4, 7, 7, 6, 5, 4, 0, 3, 3,
       7, 4, 3, 2, 6, 6, 7, 3, 4, 5, 1, 1, 0, 4], dtype=uint32), marker_cell_types={}, marker_sizes={}, markers={}, material_colors={'cube_material': '#FF7F50'}, material_data={'cube_material': {'diffuse': array([1.  , 0.5 , 0.31], dtype=float32), 'shininess': array([32.], dtype=float32), 'specular': array([0.5, 0.5, 0.5], dtype=float32)}}, material_file={'$ref': '85f25db98fe0bf20', 'ext': '.mtl', '_bytes': b'# Wavefront MTL material file\n# Material for textured cube mesh\n\nnewmtl cube_material\nKa 0.1 0.1 0.1\nKd 1.0 0.5 0.31\nKs 0.5 0.5 0.5\nNs 32.0\nd 1.0\nillum 2\n\nnewmtl secondary_material\nKa 0.1 0.1 0.1\nKd 0.2 0.8 0.2\nKs 0.3 0.3 0.3\nNs 16.0\nd 1.0\nillum 2\n'}, material_name='cube_material', materials={'cube_material': MaterialProperties(diffuse=array([1.  , 0.5 , 

## 5. Loading the Mesh from a Zip File

The Mesh class provides a class method for loading a mesh from a zip file.

In [7]:
# Load the mesh from the zip file
loaded_mesh = TexturedMesh.load_from_zip(zip_path)
print(f"Loaded mesh: {loaded_mesh.vertex_count} vertices, {loaded_mesh.index_count} indices")
print(f"Material name: {loaded_mesh.material_name}")
print(f"Tags: {loaded_mesh.tags}")

# Verify that the texture coordinates and normals were loaded correctly
print(f"\nTexture coordinates shape: {loaded_mesh.texture_coords.shape}")
print(f"Normals shape: {loaded_mesh.normals.shape}")
print(f"Material data: {loaded_mesh.material_data}")
print(f"Material colors: {loaded_mesh.material_colors}")

# Verify the dict[str, BaseModel] edge case was loaded correctly
print("\n--- BaseModel dict edge case ---")
print(f"Materials keys: {list(loaded_mesh.materials.keys())}")
for mat_name, mat in loaded_mesh.materials.items():
    print(f"  {mat_name}:")
    print(f"    type: {type(mat).__name__}")
    print(f"    diffuse: {mat.diffuse}")
    print(f"    specular: {mat.specular}")
    print(f"    shininess: {mat.shininess}")


Loaded mesh: 8 vertices, 36 indices
Material name: cube_material
Tags: ['cube', 'example']

Texture coordinates shape: (8, 2)
Normals shape: (8, 3)
Material data: {'cube_material': {'diffuse': array([1.  , 0.5 , 0.31], dtype=float32), 'shininess': array([32.], dtype=float32), 'specular': array([0.5, 0.5, 0.5], dtype=float32)}}
Material colors: {'cube_material': '#FF7F50'}

--- BaseModel dict edge case ---
Materials keys: ['cube_material', 'secondary_material']
  cube_material:
    type: MaterialProperties
    diffuse: [1.   0.5  0.31]
    specular: [0.5 0.5 0.5]
    shininess: 32.0
  secondary_material:
    type: MaterialProperties
    diffuse: [0.2 0.8 0.2]
    specular: [0.3 0.3 0.3]
    shininess: 16.0


## 6. Creating a Different Mesh Subclass

Let's create another mesh subclass with different attributes to demonstrate the flexibility of the Pydantic-based Mesh class.

In [8]:
class SkinnedMesh(Mesh):
    """
    A mesh with skinning information for animation.
    """
    # Add bone weights and indices as additional numpy arrays
    bone_weights: np.ndarray = Field(..., description="Bone weights for each vertex")
    bone_indices: np.ndarray = Field(..., description="Bone indices for each vertex")

    # Add non-array attributes
    skeleton_name: str = Field("default", description="Skeleton name")
    animation_names: list[str] = Field(default_factory=list, description="Animation names")

# Create a simple skinned mesh
skinned_mesh = SkinnedMesh(
    vertices=vertices,
    indices=indices,
    bone_weights=np.random.random((len(vertices), 4)).astype(np.float32),  # 4 weights per vertex
    bone_indices=np.random.randint(0, 4, (len(vertices), 4)).astype(np.uint8),  # 4 bone indices per vertex
    skeleton_name="human_skeleton",
    animation_names=["walk", "run", "jump"]
)

print(f"Skinned mesh created with {skinned_mesh.vertex_count} vertices and {skinned_mesh.index_count} indices")
print(f"Skeleton name: {skinned_mesh.skeleton_name}")
print(f"Animation names: {skinned_mesh.animation_names}")
print(f"Bone weights shape: {skinned_mesh.bone_weights.shape}")
print(f"Bone indices shape: {skinned_mesh.bone_indices.shape}")

Skinned mesh created with 8 vertices and 36 indices
Skeleton name: human_skeleton
Animation names: ['walk', 'run', 'jump']
Bone weights shape: (8, 4)
Bone indices shape: (8, 4)


## 7. Saving and Loading the Skinned Mesh

Let's save and load the skinned mesh to demonstrate that all attributes are preserved.

In [9]:
# Save the skinned mesh to a zip file
skinned_zip_path = Path("skinned_cube.zip")
skinned_mesh.save_to_zip(skinned_zip_path)
print(f"Saved skinned mesh to {skinned_zip_path}, file size: {skinned_zip_path.stat().st_size} bytes")

# Load the skinned mesh from the zip file
loaded_skinned_mesh = SkinnedMesh.load_from_zip(skinned_zip_path)
print(f"\nLoaded skinned mesh: {loaded_skinned_mesh.vertex_count} vertices, {loaded_skinned_mesh.index_count} indices")
print(f"Skeleton name: {loaded_skinned_mesh.skeleton_name}")
print(f"Animation names: {loaded_skinned_mesh.animation_names}")
print(f"Bone weights shape: {loaded_skinned_mesh.bone_weights.shape}")
print(f"Bone indices shape: {loaded_skinned_mesh.bone_indices.shape}")

ValueError: Array dtype itemsize must be a multiple of 4, got 1 (dtype=uint8). Use float32, int32, float64, or int64.

## 8. Cleaning Up (Part 1)

Let's clean up the files we created so far before the cache examples.

In [ ]:
# Clean up
for path in [zip_path, skinned_zip_path]:
    if Path(path).exists():
        Path(path).unlink()
        print(f"Removed {path}")

print("\nExample completed successfully!")

Removed textured_cube.zip
Removed skinned_cube.zip

Example completed successfully!


## 9. Using Callbacks for Nested Packables

When working with meshes that contain nested Packables (like our `TexturedMesh` with `PhysicsProperties`), you can use callbacks to implement custom caching, storage, or deduplication strategies.

The callback types are:
- **`on_packable` (save)**: `Callable[[Packable, str], None]` - called with `(packable, checksum)` when saving
- **`on_packable` (load)**: `Callable[[Type[Packable], str], Optional[Packable]]` - called with `(packable_type, checksum)` when loading; return `None` to fall back to embedded data

### Cache Deduplication Example

When multiple meshes share the same nested Packable data, the cache automatically deduplicates them using SHA256 hashes.

## 10. Jax Conversion Example

In [ ]:
try:
    import jax

    jax_skinned_mesh = skinned_mesh.convert_to("jax")
    print(f"Converted skinned mesh to JAX arrays, vertex dtype: {jax_skinned_mesh.vertices.dtype}")
except ImportError:
    print("JAX not available - skipping conversion example")

Converted skinned mesh to JAX arrays, vertex dtype: float32


## Example Complete!

This notebook demonstrated:
- Creating custom Mesh subclasses with additional numpy arrays
- Working with nested dictionaries containing arrays
- Using BaseModel instances with arrays inside dictionaries